In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is version 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.1.3
3.4.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation2 import get_model_for_problem_formulation, sum_over, sum_over_time

In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-4
# each problem formulation has its own list of outcomes
# 0: is our problem formulation (Veluwe); 1: different problem formulation with 2 kinds of costs together for Veluwe.
# 2: problem formulation Zutphen, 3: problem formulation Doesburg & Cortenoever; 4: problem formulation Overijssel
dike_model, planning_steps = get_model_for_problem_formulation(0)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage'), function=<function sum_over at 0x1461e3740>)
ScalarOutcome('RfR Total Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x1461e3740>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x1461e3740>)


In [8]:
# running the model through EMA workbench
with (MultiprocessingEvaluator(dike_model) as evaluator):
    results= evaluator.perform_experiments(scenarios=1000, policies=1)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1000 scenarios * 275 policies * 1 model(s) = 275000 experiments
100%|████████████████████████████████| 275000/275000 [2:59:14<00:00, 25.57it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


SyntaxError: invalid syntax (3565794782.py, line 1)

In [27]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['Expected Annual Damage', 'RfR Total Costs', 'Dike Investment Costs'])
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1133, in __repr__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1292, in to_string
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/io/formats/format.py", line 608, in __init__
  File "/Use

In [28]:
experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'],
      dtype='object')

In [29]:
# only works because we have scalar outcomes
outcomes_pd = pd.DataFrame(outcomes)
outcomes_pd

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1133, in __repr__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1310, in to_string
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/io/formats/format.py", line 1097, in to_string
ModuleNotF

In [30]:
tot = experiments.join(outcomes_pd)
tot


Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1133, in __repr__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1292, in to_string
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/io/formats/format.py", line 608, in __init__
  File "/Use

In [31]:
from ema_workbench.analysis import pairs_plotting

fig, axes = pairs_plotting.pairs_scatter(tot, outcomes, group_by='policy',legend=False)
fig.set_figheight(15)
fig.set_figwidth(15)
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/var/folders/cq/_rbxzkq17533m5qdt1c__dyw0000gn/T/ipykernel_6141/2548029488.py", line 1, in <module>
    from ema_workbench.analysis import pairs_plotting
ModuleNotFoundError: No module named 'ema_workbench.analysis'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
  File "/Users/evalievanoijen

In [32]:
tot_policy = tot.groupby('policy').mean(numeric_only=True)
tot_policy.sort_values(by=tot_policy.columns[-1], ascending=True, inplace=True)
tot_policy

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


/var/folders/cq/_rbxzkq17533m5qdt1c__dyw0000gn/T/ipykernel_6141/255886294.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tot_policy = tot.groupby('policy').mean(numeric_only=True)
Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Download

In [33]:
# outcomes to dataframe
outcomes_df = pd.DataFrame.from_dict(outcomes)

# right columns
columns = ["RfR Total Costs", "Expected Annual Damage", "Dike Investment Costs"]

# calculate mean, max, min
summary = outcomes_df[columns].agg(['mean', 'max', 'min'])

# calculate in millions
summary.loc[:, "RfR Total Costs"] = summary["RfR Total Costs"] / 1e6
summary.loc[:, "Expected Annual Damage"] = summary["Expected Annual Damage"] / 1e6
summary.loc[:, "Dike Investment Costs"] = summary["Dike Investment Costs"] / 1e6

# round to two decimals
summary = summary.round(2)

# change index names
summary.index = ['Mean', 'Maximum', 'Minimum']
summary

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1133, in __repr__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1310, in to_string
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/io/formats/format.py", line 1097, in to_string
ModuleNotF

In [19]:
summary

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 282, in catch_format_error
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/formatters.py", line 770, in __call__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 419, in pretty
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/lib/pretty.py", line 794, in _repr_pprint
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1133, in __repr__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/core/frame.py", line 1310, in to_string
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/pandas/io/formats/format.py", line 1097, in to_string
ModuleNotF

In [20]:
# # defining specific policies
# # for example, policy 1 is about extra protection in upper boundary
# # policy 2 is about extra protection in lower boundary
# # policy 3 is extra protection in random locations
#
#
# def get_do_nothing_dict():
#     return {l.name: 0 for l in dike_model.levers}
#
#
# policies = [
#     Policy(
#         "policy 1",
#         **dict(
#             get_do_nothing_dict(),
#             **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
#         )
#     ),
#     Policy(
#         "policy 2",
#         **dict(
#             get_do_nothing_dict(),
#             **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
#         )
#     ),
#     Policy(
#         "policy 3",
#         **dict(
#             get_do_nothing_dict(),
#             **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
#         )
#     ),
# ]

In [21]:
# # pass the policies list to EMA workbench experiment runs
# n_scenarios = 100
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(n_scenarios, policies)

In [22]:
# experiments, outcomes = results

In [23]:
# only works because we have scalar outcomes
# pd.DataFrame(outcomes)

In [24]:
sns.pairplot(pd.DataFrame.from_dict(outcomes))
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/var/folders/cq/_rbxzkq17533m5qdt1c__dyw0000gn/T/ipykernel_6141/500585219.py", line 1, in <module>
    sns.pairplot(pd.DataFrame.from_dict(outcomes))
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/seaborn/axisgrid.py", line 2119, in pairplot
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/seaborn/axisgrid.py", line 1280, in __init__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 1042, in figure
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 551, in new_figure_manager
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 528, in _warn_if

In [25]:
policies = experiments['policy']

data = pd.DataFrame.from_dict(outcomes)
data['policy'] = policies

sns.pairplot(data, hue='policy',  vars=outcomes.keys())
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/var/folders/cq/_rbxzkq17533m5qdt1c__dyw0000gn/T/ipykernel_6141/4030707497.py", line 6, in <module>
    sns.pairplot(data, hue='policy',  vars=outcomes.keys())
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/seaborn/axisgrid.py", line 2119, in pairplot
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/seaborn/axisgrid.py", line 1280, in __init__
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 1042, in figure
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 551, in new_figure_manager
  File "/Users/evalievanoijen/Downloads/EPA133a-Lab/venv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 528, i

In [39]:
from ema_workbench import save_results


save_results(results, 'dike_open_exploration_results.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/evalievanoijen/PycharmProjects/group25_mbdm/final assignment/dike_open_exploration_results.tar.gz
